<a href="https://colab.research.google.com/github/GrzegorzMeller/AlgorithmsForMassiveData/blob/master/OCULAR_DISEASE_RECOGNITION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
!kaggle datasets download -d andrewmvd/ocular-disease-recognition-odir5k
!unzip ocular-disease-recognition-odir5k.zip

 98% 1.23G/1.26G [00:16<00:00, 42.7MB/s]
100% 1.26G/1.26G [00:16<00:00, 82.0MB/s]


In [0]:
import tensorflow as tf
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [0]:
BATCH_SIZE = 32
IMG_HEIGHT = 224
IMG_WIDTH = 224

training_images_file = 'ODIR-5K/Training_Images'
testing_images_file = 'ODIR-5K/Testing_Images'
labels_file = 'ODIR-5K/data.xlsx'

In [30]:
#load labels to pandas
import pandas as pd
labels = pd.read_excel(labels_file, index_col=0) 
labels['Left-Diagnostic Keywords'][0]

'cataract'

In [0]:
list_ds = tf.data.Dataset.list_files("ODIR-5K/Training_Images/*.jpg")

In [22]:
for f in list_ds.take(5):
  print(f.numpy())

b'ODIR-5K/Training_Images/2505_left.jpg'
b'ODIR-5K/Training_Images/2774_left.jpg'
b'ODIR-5K/Training_Images/492_left.jpg'
b'ODIR-5K/Training_Images/12_right.jpg'
b'ODIR-5K/Training_Images/4126_left.jpg'


In [34]:
def get_label(file_path):
  u_id = str(file_path).split("/")[2].split("_")[0]
  label = ""
  if labels['Left-Diagnostic Keywords'][int(u_id)] == "normal fundus":
    label = "N"
  elif labels['D'][int(u_id)] == 1:
    label = "D"
  elif labels['G'][int(u_id)] == 1:
    label = "G"
  elif labels['C'][int(u_id)] == 1:
    label = "C"
  elif labels['A'][int(u_id)] == 1:
    label = "A"
  elif labels['H'][int(u_id)] == 1:
    label = "H"
  elif labels['M'][int(u_id)] == 1:
    label = "M"
  elif labels['O'][int(u_id)] == 1:
    label = "O"
  return label

#get_label("ODIR-5K/Training_Images/0_left.jpg")

'C'